In [0]:
%pip install kaggle azure-storage-blob

  Obtaining dependency information for kaggle from https://files.pythonhosted.org/packages/75/f6/cccedb8db42beac1c5d7cfa57d5b800b90ef91118833972af3f7b5e159c8/kaggle-1.7.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for bleach from https://files.pythonhosted.org/packages/fc/55/96142937f66150805c25c4d0f31ee4132fd33497753400734f9dfdcbdc66/bleach-6.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for python-slugify from https://files.pythonhosted.org/packages/a4/62/02da182e544a51a5c3ccf4b03ab79df279f9c60c5e82d5e8bec7ca26ac11/python_slugify-8.0.4-py2.py3-none-any.whl.metadata
  Obtaining dependency information for text-unidecode from https://files.pythonhosted.org/packages/a6/a5/c0b6468d3824fe3fde30dbb5e1f687b291608f9473681bbf7dabbf5a87d7/text_unidecode-1.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762aeb1c1184ed1f6b14/tqdm-4.67.1-py3-

In [0]:
import os
from azure.storage.blob import BlobServiceClient

In [0]:
# Assign the authentication details prior to importing the Kaggle library because it authenticates at the time of import.
os.environ['KAGGLE_USERNAME'] = 'naveengandu'
os.environ['KAGGLE_KEY'] = '407179e1a8def05b1b8baf22198b993a'

In [0]:
# Import the kaggle library
from kaggle.api.kaggle_api_extended import KaggleApi

In [0]:
# Initialize Kaggle API
api = KaggleApi()
api.authenticate()

In [0]:
# Azure Data Lake Storage credentials
# Hardcoding these keys isn't the best practice due to potential security risks. However, for the sake of simplicity and limitations of the trial databricks subscription, this method was followed.
STORAGE_ACCOUNT_NAME = "projectrouge6363905707"
STORAGE_ACCOUNT_KEY = "Mrth7ayFyx5DPRYkTmtMwRFrq0IU9WQuB/xCFsNbn/ApJOTP1m66DM5/M8BSikzUPMVTmxPcIRgc+AStAPeDdg=="
CONTAINER_NAME = "rawkaggledata"

In [0]:
# Connect to Azure Blob Storage (ADLS Gen2)
blob_service_client = BlobServiceClient(account_url=f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net",
                                        credential=STORAGE_ACCOUNT_KEY)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

In [0]:
# Define dataset

# - Note to self - probably best to arrange the list in order of increasing dataset size and parallelize. This is 
# executed on a single node but might be more cost effective and efficient to run this on multiple nodes due to the huge 
# dataset sizes and the nested for loops. 

# - The job crashed the first time I ran it, so I rearranged the order of the datasets. 

# - The dataset ("minhhuy2810/rice-diseases-image-dataset") is too large to download in one go and so running it always 
# displays an error - "Fatal error: The Python kernel is unresponsive.". So removing it from the list of datasets to be 
# downloaded. A more powerful cluster would be required to download the entire dataset. 

datasets = ["vbookshelf/rice-leaf-diseases", "maimunulkjisan/rice-leaf-dataset-from-mendeley-data", "anshulm257/rice-disease-dataset", "minhhuy2810/rice-diseases-image-dataset"]

In [0]:
import shutil

# Download the entire dataset to the temporary directory
print("Downloading dataset from Kaggle...")

for dataset in datasets:
    # Create a temporary directory in Databricks
    temp_dir = "/dbfs/tmp/kaggle_datasets/"
    os.makedirs(temp_dir, exist_ok=True)

    api.dataset_download_files(dataset, path=temp_dir, unzip=True)

    # Upload each file to Azure Data Lake
    for root, _, files in os.walk(temp_dir):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            blob_path = os.path.relpath(file_path, temp_dir)  # Preserve folder structure

            print(f"Uploading {blob_path} to Azure Data Lake...")

            # Read file in binary mode
            with open(file_path, "rb") as data:
                blob_client = container_client.get_blob_client(blob_path)
                blob_client.upload_blob(data, overwrite=True)
            
    # Clean up local files
    shutil.rmtree(temp_dir)
    print(f"{dataset} uploaded successfully and temporary files deleted.")

print("All datasets uploaded successfully.")


---------------------------------------------------------------------------
The Python process exited with exit code 137 (SIGKILL: Killed). This may have been caused by an OOM error. Check your command's memory usage.



The last 10 KB of the process's stderr and stdout can be found below. See driver logs for full logs.
---------------------------------------------------------------------------
Last messages on stderr:
Wed Mar 19 11:32:34 2025 Connection to spark from PID  19291
Wed Mar 19 11:32:34 2025 Initialized gateway on port 43963
Wed Mar 19 11:32:34 2025 Connected to spark.
2025/03/19 11:32:37 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of langchain. If you encounter errors during autologging, try upgrading / downgrading langchain to a supported version, or try upgrading MLflow.
2025/03/19 11:32:38 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of openai. If you encounter errors during autologging, try upgrading / do

In [0]:
# Clean up local files
import shutil

temp_dir = "/dbfs/tmp/kaggle_datasets/"
shutil.rmtree(temp_dir)
print("✅ All datasets uploaded successfully and temporary files deleted.")

✅ All datasets uploaded successfully and temporary files deleted.


In [0]:
# verify that the dataset is available in the specified storage account/container.
spark.conf.set(
    f"fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
    f"{STORAGE_ACCOUNT_KEY}"
)

display(dbutils.fs.ls(f"abfss://{CONTAINER_NAME}@{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/"))


path,name,size,modificationTime
abfss://rawkaggledata@projectrouge6363905707.dfs.core.windows.net/Rice Leaf Disease Images/,Rice Leaf Disease Images/,0,1742385710000
abfss://rawkaggledata@projectrouge6363905707.dfs.core.windows.net/Rice_Leaf_AUG/,Rice_Leaf_AUG/,0,1742386506000
abfss://rawkaggledata@projectrouge6363905707.dfs.core.windows.net/rice_leaf_diseases/,rice_leaf_diseases/,0,1742383328000
